# RLC series circuit

The purpose of this study is to curve fit ring down data to a RLC circuit and solve for the component values.  Fire pulse data into a CVR will be looked at to see how close the circuits match.  The following items will be investigated.
- statistical look at the fire pulse data from AP and QT
- equivalent circuits
- learn mark down and equation solving in python

The Series RLC Circuit
----------------------
The loop equation for the circuit is: $L\frac{di}{dt} + Ri + \frac{1}{C}\int_{0}^{i} idt + Vo = 0$

The characteristic equation is: $Ls^{2} + Rs + \frac{1}{C} = 0$

The natural frequencies are: $s_{1,2} = -\frac{R}{2L}\pm\sqrt{(\frac{R}{2L})^2-\frac{1}{LC}}$

The circuit is under damped if: $C \lt \frac{4L}{R^2}$

The resonant frequency is: $\omega _d = \frac{1}{\sqrt{LC}}$

The damping coefficient is: $\alpha = \frac{R}{2L}$

The damped frequency is: $\omega _d = \sqrt{\omega _{0}^{2}-\alpha ^{2}}$

The underdamped responce is: $i = e^{-\alpha t}(A_{1}cos\omega _d t + A_{2}sin\omega _d t)$

Reference: page 227, Basic Electric Circuit Analysis, David E. Johnson, John L. Hibum, Johnny R. Johnson, 1978

The following cells walk through the derivation of the equations with python

See package at

https://github.com/mph-/lcapy

http://lcapy.elec.canterbury.ac.nz/

http://firsttimeprogrammer.blogspot.com/2015/12/step-response-of-rlc-series-circuit.html



In [13]:
from sympy import *
from pylab import *
init_printing()

In [2]:
# create element symbols
R, L, C = symbols('R L C')
# create voltage and current symbols
i, v, Vo = symbols('i, v, Vo')
t = symbols('t',positive=True)  # t > 0

In [3]:
# loop current for a series R, L & C circuit, i(t)
exp = i*R + L*Derivative(i,t) + Integral(i,(t,0,i))/C + Vo
exp

In [4]:
# doing some of the math by hand, taking the derivative to elimate the integral
#writing the current as a function of t
f = Function('f')  # Function as a constructor
exp = Eq(L*Derivative(f(t),t,t) + R*Derivative(f(t),t) + f(t)/C, -Vo)
exp

In [5]:
#solving the differential equation
dsolve(exp, f(t))

In [6]:
#substitute some values from the example
# values from the ring down tests: L=31.8nH, R=.155ohm, C=0.1uF
exp = exp.subs({L:1})
exp = exp.subs({R:1})
exp = exp.subs({C:.1})
exp = exp.subs({Vo:10})
exp

In [8]:
# Solve an ordinary differential equation following the example
exp_a = dsolve(exp, f(t))
exp_a

In [12]:
# plot some data
time = linspace(0.0, 0.6e-6, 1001)

vc, x = odeint(rlc,[0.0,0.0],time).T
i = 1.0e-9*x

plt.plot(time, vc)
plt.show()

plot some data
do a lease squares fit to:

The damping coefficient is: $\alpha = \frac{R}{2L}$

The damped frequency is: $\omega _d = \sqrt{\omega _{0}^{2}-\alpha ^{2}}$


writing the characteristic equation in terms of s
-------------------------------------------------

In [24]:
from sympy import inverse_laplace_transform as ilt

In [25]:
# create the s variable
R, L, C = symbols('R L C')
s = Symbol('s')
t = Symbol('t',positive=True)  # t > 0

In [26]:
# this is the characteristice equation
equ1 = L*s**2 + R*s + 1/C
equ1

In [27]:
roots(equ1,s)

In [28]:
# write an expression for the current through the resistor
equ1a = (s/L)/(s**2 + s*R/L + 1/(L*C))
equ1a

In [46]:
equ1a = equ1a.subs({L:1})
equ1a = equ1a.subs({R:1})
equ1a = equ1a.subs({C:1})
equ1a

In [47]:
equ2a = ilt(equ1a, s, t)
equ2a

generate some data and do a 2nd order curve fit

In [54]:
"""
The following example copied from the web
need to figure out how to do this in sympy
"""
from scipy.integrate import odeint
#import matplotlib.pyplot as plt
from pylab import *

def rlc(A,t):
    Vc, x = A
    V = 1.0 #voltage source
    R = 5.0
    L = 100.0e-9
    C = 1.0e-9
    res = array([x, (V-Vc-(x*R*C))/(L*C)])
    return res

time = linspace(0.0, 0.6e-6, 1001)
vc, x = odeint(rlc,[0.0,0.0],time).T
i = 1.0e-9*x

plt.plot(time, vc)
plt.show()
